## Model training

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator as IDG
import numpy as np
import imp
import utils; imp.reload(utils) 
from utils import plot_batch
from utils import Vgg16BN

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


RuntimeError: 
Could not import 'mkl'.  Either install mkl-service with conda or set
MKL_THREADING_LAYER=GNU in your environment for MKL 2018.

If you have MKL 2017 install and are not in a conda environment you
can set the Theano flag blas.check_openmp to False.  Be warned that if
you set this flag and don't set the appropriate environment or make
sure you have the right version you *will* get wrong results.


In [ ]:
data_path="../data/"
train_dir="/train"
val_dir="/val"
test_dir="/test"
project_name="statefarm"
project_data_path=data_path+project_name
train_data_path=project_data_path+train_dir
val_data_path=project_data_path+val_dir
test_data_path=project_data_path+test_dir
weights_file = project_data_path+'/statefarm.h5'

In [ ]:
# idg = IDG(rotation_range=15, height_shift_range=0.05, 
#                 shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

idg = IDG(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

In [ ]:
classes = ["".join("c"+str(n)) for n in range(0,10)]
classes

In [ ]:
class_mode = 'categorical'
batch_size = 64
train_gen = idg.flow_from_directory(train_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)
val_gen = idg.flow_from_directory(val_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)

In [ ]:
num_of_samples = len(train_gen.filenames)
print(num_of_samples)

In [ ]:
train_batch, train_labels = train_gen.next()
val_batch, val_labels = val_gen.next()

In [2]:
print(train_data_path)
print(val_data_path)
print(classes)

NameError: name 'train_data_path' is not defined

In [3]:
plot_batch(val_batch, val_labels, classes)

NameError: name 'plot_batch' is not defined

In [4]:
print(train_batch.shape)
print(val_batch.shape)

NameError: name 'train_batch' is not defined

In [11]:
from keras.applications.vgg16 import VGG16

In [12]:
image_shape = train_batch.shape[1:]
vgg16_base = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=image_shape, pooling=None)

58769408/58889256 [============================>.] - ETA: 0s

In [13]:
vgg16_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [14]:
from keras.layers import Flatten, Dense, BatchNormalization, Dropout
from keras.models import Model

vgg16_top = vgg16_base.output
vgg16_top = Flatten()(vgg16_top)
# let's add a fully-connected layer
vgg16_top = Dense(1024, activation='relu')(vgg16_top)
vgg16_top = BatchNormalization()(vgg16_top)
vgg16_top = Dropout(0.5)(vgg16_top)
vgg16_top = Dense(1024, activation='relu')(vgg16_top)
vgg16_top = BatchNormalization()(vgg16_top)
vgg16_top = Dropout(0.5)(vgg16_top)
# and a logistic layer -- let's say we have 2 classes
vgg16_top = Dense(10, activation='softmax')(vgg16_top)

# this is the model we will train
model = Model(inputs=vgg16_base.input, outputs=vgg16_top)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in vgg16_base.layers:
    layer.trainable = False

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [16]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

In [17]:
#train_gen = idg.flow_from_directory(train_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)
#val_gen = idg.flow_from_directory(val_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)

In [21]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
num_batches = num_of_samples/batch_size
epochs = 10
steps_per_epoch = 17880./batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=3, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2, verbose=1, mode='auto')
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto', period=1)

In [ ]:
# fine-tune the model
model.fit_generator(train_gen, steps_per_epoch=num_batches, epochs=epochs, verbose=1,
                    callbacks=[reduce_lr, early_stopping, model_checkpoint],
                    validation_data=val_gen, validation_steps=1) 

Epoch 1/10


In [19]:
model.load_weights(weights_file)

In [20]:
class_mode=None
batch_size=64
steps=np.ceil(4481./batch_size)
val_gen = idg.flow_from_directory(val_data_path, batch_size=batch_size, shuffle=False, class_mode=class_mode, classes=classes)
prob_val=model.predict_generator(val_gen, steps, verbose=True)

Found 4544 images belonging to 10 classes.
71/71 [==============================] - 74s    


In [21]:
from sklearn.metrics import confusion_matrix
y_pred=np.argmax(prob_val,axis=1)
y_true=val_gen.classes
confusion_matrix(y_true,y_pred)

array([[318,  19,   5,  14,  20,   2,   1,  20,  25,  51],
       [ 73, 284,   0,  12,   0,   7,  21,  43,  57,   2],
       [ 34,  12, 327,   2,   4,   3,   8,  26,  68,   3],
       [ 20,   0,   5, 359,  12,  12,   0,  44,  31,   2],
       [ 12,   0,   6,  10, 306,  10,   1,  37,  99,  16],
       [  1,   2,   0,   3,   0, 457,   1,   0,  20,  10],
       [ 17,  20,  10,   6,  12,   4, 339,  16,  43,   0],
       [  7,   1,   0,   0,   0,   4,   1, 372,  20,   5],
       [ 38,  10,  19,  14,   5,  10,   3,  25, 223,  11],
       [ 99,   2,   7,  10,   7,   8,   0,  34,  29, 176]])

In [23]:
test_gen=idg.flow_from_directory(test_data_path, batch_size=batch_size,
                                   class_mode=None,  # only data, no labels
                                   shuffle=False)  # keep data in same order as labels

Found 79726 images belonging to 1 classes.


In [24]:
batch_size=128
test_dataset_size = len(test_gen.filenames)
num_of_batches = test_dataset_size/batch_size

In [25]:
prob_test=model.predict_generator(test_gen, num_of_batches, verbose=True)

623/622 [==============================] - 1332s  


In [26]:
import pandas as pd
n1 = [s.split('/')[1] for s in test_gen.filenames]
# n2 = [(s.split('.')[0]) for s in n1]
results = pd.DataFrame(prob_test)
temp = pd.DataFrame(np.array(n1))
results=pd.concat([temp, results],axis=1)
results.tail()

,0,0,1,2,3,4,5,6,7,8,9
79721,img_41337.jpg,1.725254e-09,4.807308e-09,0.000012,3.138028e-10,6.435936e-08,1.320339e-08,1.618547e-10,0.997730,0.000002,0.002257
79722,img_33426.jpg,4.919239e-07,6.172560e-04,0.000033,1.068246e-04,1.416290e-07,7.997638e-07,9.991979e-01,0.000013,0.000029,0.000001
79723,img_98788.jpg,1.908796e-04,5.736105e-04,0.000898,5.328410e-06,1.851416e-04,6.592942e-06,2.561159e-05,0.000056,0.998056,0.000003
79724,img_18736.jpg,9.352832e-01,3.597283e-04,0.009242,1.720109e-03,3.411957e-02,3.894767e-03,5.912402e-04,0.002552,0.011789,0.000449
79725,img_40754.jpg,2.502447e-02,2.735717e-03,0.000013,6.256847e-03,2.410381e-04,6.458788e-04,6.289176e-05,0.060301,0.006718,0.898002


In [27]:
results.to_csv('test.csv')
from IPython.display import FileLink
FileLink('test.csv')

/src/workspace/exercises/test.csv

## VGG16 with batchnormalization

In [21]:
vgg16=Vgg16BN(size=(256,256,3))

In [22]:
vgg16.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 256, 256, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
__________

In [23]:
from keras import optimizers
vgg16.model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
weights_file = project_data_path+'/statefarm_bn.h5'

In [24]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
num_of_samples = len(train_gen.filenames)
num_batches = num_of_samples/batch_size
epochs = 10
steps_per_epoch = batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=3, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2, verbose=0, mode='auto')
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto', period=1)

In [ ]:
class_mode = 'categorical'
batch_size = 128
train_gen = idg.flow_from_directory(train_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)
val_gen = idg.flow_from_directory(val_data_path, batch_size=batch_size, shuffle=True, class_mode=class_mode, classes=classes)

Found 17880 images belonging to 10 classes.
Found 4544 images belonging to 10 classes.


In [ ]:
num_batches = num_of_samples/batch_size
epochs = 10
steps_per_epoch = batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=3, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2, verbose=0, mode='auto')
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto', period=1)
vgg16.model.fit_generator(train_gen, steps_per_epoch=num_batches, epochs=epochs, 
                    callbacks=[reduce_lr, early_stopping, model_checkpoint],
                    validation_data=val_gen, validation_steps=10) 

Epoch 1/10
559/558 [==============================] - 830s - loss: 0.2761 - acc: 0.9130 - val_loss: 2.7147 - val_acc: 0.4531
Epoch 2/10
559/558 [==============================] - 831s - loss: 0.2498 - acc: 0.9207 - val_loss: 1.5767 - val_acc: 0.6406
Epoch 3/10
559/558 [==============================] - 830s - loss: 0.2179 - acc: 0.9306 - val_loss: 1.4010 - val_acc: 0.6875
Epoch 4/10
559/558 [==============================] - 831s - loss: 0.2046 - acc: 0.9353 - val_loss: 1.3958 - val_acc: 0.6344
Epoch 5/10
559/558 [==============================] - 815s - loss: 0.1802 - acc: 0.9421 - val_loss: 1.7393 - val_acc: 0.5719
Epoch 6/10
488/558 [=========================>....] - ETA: 102s - loss: 0.1317 - acc: 0.9587

In [ ]:
vgg16.model.load_weights(weights_file)

In [ ]:
class_mode=None
batch_size=64
steps=np.ceil(4481./batch_size)
val_gen = idg.flow_from_directory(val_data_path, batch_size=batch_size, shuffle=False, class_mode=class_mode, classes=classes)
prob_val_bn=vgg16.model.predict_generator(val_gen, steps, verbose=True)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_bn=np.argmax(prob_val_bn,axis=1)
y_true=val_gen.classes
confusion_matrix(y_true,y_pred_bn)

In [ ]:
test_gen=idg.flow_from_directory(ext_test_data_path, batch_size=64,
                                   class_mode=None,  # only data, no labels
                                   shuffle=False)  # keep data in same order as labels

In [20]:
test_dataset_size = len(test_gen.filenames)
num_of_batches = test_dataset_size/batch_size

In [21]:
prob_test_bn=vgg16.model.predict_generator(test_gen, num_of_batches, verbose=True)

1246/1245 [==============================] - 1315s  


In [22]:
prob_test_bn_clipped = prob_test_bn
prob_test_bn_clipped.shape

(79726, 10)

In [23]:

print(test_dataset_size)

79726


In [24]:
import pandas as pd
n1 = [s.split('/')[1] for s in test_gen.filenames]
# n2 = [(s.split('.')[0]) for s in n1]
results = pd.DataFrame(prob_test_bn_clipped)
temp = pd.DataFrame(np.array(n1))
results=pd.concat([temp, results],axis=1)
results.head()

,0,0,1,2,3,4,5,6,7,8,9
0,img_46339.jpg,5.199347e-05,3.281236e-05,0.001884,0.001500,0.000492,9.910320e-01,4.383707e-04,0.000425,4.136424e-03,7.952717e-06
1,img_18865.jpg,1.159507e-07,8.771667e-01,0.000038,0.012156,0.067809,2.150468e-03,4.052979e-02,0.000021,1.081403e-04,2.090742e-05
2,img_23088.jpg,1.131414e-03,5.743243e-07,0.000001,0.000039,0.000003,9.988142e-01,1.232827e-07,0.000003,9.643048e-07,7.071586e-06
3,img_92913.jpg,7.731178e-06,6.973586e-04,0.019066,0.002917,0.021448,2.133657e-04,5.037335e-01,0.007926,4.439891e-01,1.905155e-06
4,img_82508.jpg,1.319885e-09,9.604002e-06,0.000074,0.000002,0.000011,2.540226e-07,6.827580e-01,0.317024,1.211051e-04,3.813770e-09


In [25]:
results.to_csv('test.csv')

In [26]:
from IPython.display import FileLink
FileLink('test.csv')

/src/workspace/exercises/test.csv